<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

In [1]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer
import time
from confluent_kafka.admin import AdminClient, NewTopic


import time
#import qcutils

servers="broker:29092"

sr_url="http://schema-registry:8081"

topic = 'lab-avro-producer'

admin_client = AdminClient({
    "bootstrap.servers": servers
})




In [2]:
topic_list = []
topic_list.append(NewTopic(topic, 1, 1))
admin_client.create_topics(topic_list)

{'lab-avro-producer': <Future at 0x7fbdd8814790 state=running>}

In [3]:
producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }

# write a kafka avro producer

Define avro schemas for keys and value to describe cars

### Key
The key has a single field to contain `Plate`

### Value
A car is described by `Manifacturer`, `Model`, `Year`

1) define the key schema

In [6]:
key_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "CarKey",
  "fields": [
    {
      "name": "plate",
      "type": "string"
    }
  ]
}
"""
key_schema = avro.loads(key_schema_str)

2) define the value schema

In [7]:
value_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "CarValue",
  "fields": [
    {
      "name": "manifacturer",
      "type": "string"
    },
    {
      "name": "model",
      "type": "string"
    },
    {
      "name": "year",
      "type": "int"
    }
  ]
}
"""
value_schema = avro.loads(value_schema_str)

3) create a topic `<surname>-avro-cars`, send in messages and read it put with a consumer (can you re-use the previous one?)

In [8]:
ap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema)

In [9]:
key = {"plate": "ab123cd"}
value = {"manifacturer": "audi", "model": "A1", "year":2018}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)

print(value)

key = {"plate": "ab321cd"}
value = {"manifacturer": "skoda", "model": "karoq", "year":2018}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)

print(value)

ap.flush(1)

{'manifacturer': 'audi', 'model': 'A1', 'year': 2018}
{'manifacturer': 'skoda', 'model': 'karoq', 'year': 2018}


0

## Consume the data

In another notebook, Write a AVRO Consumer that consume the car messages and access directly to the production year...

## Schema Evolution and Value restriction

The different manifacturers added the `Color` to the car description and restricted the number of available colors to 3 (Black, White and Red).

**HINT**: check [AVRO modelling language](https://avro.apache.org/docs/current/idl.html)

Change the schema and write the original topic.

In [17]:
value_schema_str = """
{
  "namespace": "lab.avro",
  "type": "record",
  "name": "CarValue",
  "fields": [
    {
      "name": "manifacturer",
      "type": "string"
    },
    {
      "name": "model",
      "type": "string"
    },
    {
      "name": "year",
      "type": "int"
    },
    {
      "name": "color",
      "type": {
        "type": "enum",
        "name": "possibleColors",
        "symbols": ["no_color","black", "white", "red"]
      },
      "default": "no_color",
      "doc": "The color of the car"
    }
  ]
}
"""

value_schema = avro.loads(value_schema_str)
key_schema = avro.loads(key_schema_str)

producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }
        
newap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema)

key = {"plate": "ab567cd"}
value = {"manifacturer": "fiat", "model": "panda", "year":2020, "color":"black"}
newap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)


{'manifacturer': 'fiat', 'model': 'panda', 'year': 2020, 'color': 'black'}


In [18]:
key = {"plate": "ab568cd"}
value = {"manifacturer": "fiat", "model": "panda", "year":2020}
newap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)

AvroTypeException: The datum {'manifacturer': 'fiat', 'model': 'panda', 'year': 2020} is not an example of the schema {
  "type": "record",
  "name": "CarValue",
  "namespace": "lab.avro",
  "fields": [
    {
      "type": "string",
      "name": "manifacturer"
    },
    {
      "type": "string",
      "name": "model"
    },
    {
      "type": "int",
      "name": "year"
    },
    {
      "type": {
        "type": "enum",
        "name": "possibleColors",
        "namespace": "lab.avro",
        "symbols": [
          "no_color",
          "black",
          "white",
          "red"
        ]
      },
      "name": "color",
      "default": "no_color",
      "doc": "The color of the car"
    }
  ]
}

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.